# Baseline - Most popular
Use normalized ratings for calculation

In [18]:
import importlib
from collections import defaultdict

import pandas as pd
from sklearn.model_selection import train_test_split

import bibrec.server.Utils as Utils
import bibrec.server.evaluation as eval
from bibrec.server.Utils import assign_popular_based_score

importlib.reload(eval)
importlib.reload(Utils)

books, users, ratings = Utils.get_normalized_data()

/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=";", encoding="latin-1")
/Users/johannes/Studium/MMT/1. Semester/Fächer/Recommender Systems/Abschlussprojekt/code/recommender-systems/bibrec/server/Utils.py:98: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


Normalized Ratings Count: 383962


In [19]:
users_with_ratings = ratings["user_id"].unique()

# normal splitting since user specific splitting doesn't make that much of a difference and takes a long time
train, test = train_test_split(ratings, test_size=0.25)

In [20]:
print("Total Ratings Count:", len(train) + len(test))
print("Train Data Ratings Count:", len(train))
print("Test Data Ratings Count:", len(test))

Total Ratings Count: 383962
Train Data Ratings Count: 287971
Test Data Ratings Count: 95991


In [21]:
book_ratings_dict = {
    'user_id': train['user_id'].values,
    'isbn13': train['isbn13'].values,
    'book_rating': train['normalized_rating'].values
}

book_rating_df = train[['user_id', 'isbn13', 'normalized_rating']]

In [22]:
train["isbn13"] = train["isbn13"].astype("int")
books["isbn13"] = books["isbn13"].astype("int")
popular_books = assign_popular_based_score(train, books, "user_id", "isbn13", "normalized_rating")
popular_books = popular_books.sort_values("vote_count", ascending=False)

print(popular_books[:20])

              isbn13  vote_count  avg_rating  weighted_rating
9194   9780316666343         521    0.379098         0.377289
33517  9780971880108         432   -1.943317        -1.934795
14284  9780385504201         370    0.687186         0.682986
8042   9780312195519         291    0.506008         0.501912
4666   9780060928339         251    0.261791         0.258978
23540  9780590353427         245    0.718934         0.712351
7104   9780142001745         229    0.660638         0.654104
9176   9780316601955         217    0.025862         0.024767
1721    978044667227         215    0.436349         0.431460
20396  9780452282155         207    0.366009         0.361606
24726  9780671027360         204    0.310097         0.306173
17023  9780440237228         199    0.120899         0.118760
26806  9780679764021         196    0.106604         0.104577
28451  9780743418171         186    0.417827         0.412382
29795  9780786868711         181    0.129948         0.127500
12453  9

In [23]:
pop_dict = {
    'item_id': popular_books['isbn13'].values,
    'est_r': popular_books['avg_rating'].values,
}

pop_df = pd.DataFrame.from_dict(pop_dict)

ratings_dict = {
    'user_id': test['user_id'].values,
    'item_id': test['isbn13'].values,
    'rating': test['normalized_rating'].values
}

ratings_df = pd.DataFrame.from_dict(ratings_dict)

In [24]:
test_uids = ratings_df["user_id"].unique()

print("User Count:", len(test_uids))

User Count: 29864


In [25]:
def def_value():
    return "Not Present"


top_n = defaultdict(def_value)
for uid in test_uids:
    top_n[uid] = pop_df

In [26]:
import time

importlib.reload(eval)
importlib.reload(Utils)
start_time = time.time()
avg_precision = eval.get_avg_precision(ratings_df, top_n, k=50)

print("Average Precision:", avg_precision)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Precision: 0.0015227245483825495
--- Calculation time: 61.55460524559021 seconds ---


In [27]:
avg_recall = eval.get_avg_recall(ratings_df, top_n, k=50)

print("Average Recall:", avg_recall)
print("--- Calculation time: %s seconds ---" % (time.time() - start_time))

Average Recall: 0.0312002483568644
--- Calculation time: 127.75377702713013 seconds ---
